In [1]:
import torch.nn as nn

from datasets import *
from evaluation import *
from models import *
from pruning.unstructured import *
from pruning.structured import *
from pyJoules.device.nvidia_device import NvidiaGPUDomain

Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/unstructured/L1norm.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/Train.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/unstructured/random.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/decay_pruning.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/two_to_four.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/Thinet.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/Global_pruning.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/Ln_structured.py


In [2]:
#dataset = MNIST()
dataset = CIFAR10()
train_loader, test_loader = dataset.get_dataloader()

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#train model and return it
#model = Lenet()
#model = ResNet50_Cifar10()
model = AlexNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model = Trainer(model, 10, train_loader, criterion, optimizer)
original_model = model.train()
#print("Accuracy of original model : ", accuracy(original_model, test_loader))

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 6.4586 | 
Epoch: 2 | train_loss: 6.1673 | 
Epoch: 3 | train_loss: 6.0003 | 
Epoch: 4 | train_loss: 5.8726 | 
Epoch: 5 | train_loss: 5.7446 | 
Epoch: 6 | train_loss: 5.6102 | 
Epoch: 7 | train_loss: 5.4903 | 
Epoch: 8 | train_loss: 5.3844 | 
Epoch: 9 | train_loss: 5.2497 | 
Epoch: 10 | train_loss: 5.1259 | 


In [4]:
#for UnstructuredL1normPrune
# model = Lenet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(original_model.parameters(), lr=0.001)

#trainer = RandomUnstructured(original_model, 5, train_loader, criterion, optimizer, 0.5)
trainer = RandomUnstructured(original_model, 10, train_loader, criterion, optimizer,0.5)

#train and prune the model and return both the original and pruned model
#original_model, pruned_model = trainer.train_prune_retrain()
pruned_model = trainer.prune_model()

In [5]:
print("Accuracy of original model : ", accuracy(original_model, test_loader))
print("Accuracy of pruned model : ", accuracy(pruned_model, test_loader))

begin timestamp : 1713072849.446583; tag : accuracy; duration : 2.248340129852295; nvidia_gpu_7 : 166432
Accuracy of original model :  [70.62]
begin timestamp : 1713072851.6973245; tag : accuracy; duration : 2.0185494422912598; nvidia_gpu_7 : 126839
Accuracy of pruned model :  [11.99]


In [6]:
%%bash --bg 
while true; do nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> unpruned_output.txt; sleep 0.1; done

In [7]:
print("Accuracy of original model : ", accuracy(original_model, test_loader))

with open('unpruned_output.txt', 'a') as f:
    f.write("\energy computation for original model completed.\n")

begin timestamp : 1713072853.927874; tag : accuracy; duration : 2.060861825942993; nvidia_gpu_7 : 129122
Accuracy of original model :  [70.61]


In [8]:
%%bash
pgrep -f "watch -n 1 'nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> unpruned_output.txt'" | xargs -r kill

In [9]:
%%bash --bg 
while true; do nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> pruned_output.txt; sleep 0.1; done

In [10]:
print("Accuracy of pruned model : ", accuracy(pruned_model, test_loader))

with open('pruned_output.txt', 'a') as f:
    f.write("\energy computation for pruned model completed.\n")

begin timestamp : 1713072856.3752003; tag : accuracy; duration : 2.0146594047546387; nvidia_gpu_7 : 125811
Accuracy of pruned model :  [12.16]


In [11]:
%%bash
pgrep -f "watch -n 1 'nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> pruned_output.txt'" | xargs -r kill

In [12]:
# total = 0
# count=0
# with open('unpruned_output.txt', 'r') as f:
#     for line in f:
#         if '\\energy' in line:
#             break
#         total += float(line.strip())
#         count += 1

# average = total / count if count else 0
# print(average)



In [ ]:
# total = 0
# count=0
# with open('pruned_output.txt', 'r') as f:
#     for line in f:
#         if '\\energy' in line:
#             break
#         total += float(line.strip())
#         count += 1

# average = total / count if count else 0
# print(average)

